## 1. Retrieve the Workspace and a Previously Registered Scikit Learn Model

In [7]:
%%time
from azureml.core import Workspace, Run

ws = Workspace.from_config(path='..\\aml_config\\config.json')
print(ws.name, ws.location, ws.resource_group, sep = '\t')

Found the config file in: F:\temp\schemaTest\aml_config\config.json
trangevimaster	centraluseuap	trangevi
Wall time: 323 ms


In [6]:
%%time
from azureml.core.model import Model

model = Model(ws, 'Salary_Predictor')

## 2. Sample Scoring and Environment Files are Below

In [20]:
%%writefile score.py
import sklearn
from azureml.core.model import Model
from sklearn.externals import joblib

import numpy as np
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


input_sample = np.array([[1.8]])
output_sample = np.array([43638.88864165144])


def init():
    global model

    model_file = Model.get_model_path('Salary_Predictor')
    with open(model_file, 'rb') as f:
        model = joblib.load(f)


@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        res = model.predict(data)
        return res.tolist()
    except Exception as e:
        return str(e)

Overwriting score.py


In [9]:
%%writefile myenv.yml
name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
  
  - python=3.6.2
  - pip:
    # Required packages for AzureML execution, history, and data preparation.
    - azureml-defaults
    - scikit-learn
    - inference-schema[numpy-support]

Writing myenv.yml


## 3. Deploy the Model to ACI

In [21]:
%%time
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

from azureml.core.image import ContainerImage

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=2,
                                               tags={"data": "Salary-Data", "method":"sklearn"},
                                               description='Sample Model for salary predictions')

image_config = ContainerImage.image_configuration(execution_script="./score.py",
                                                  runtime="python",
                                                  conda_file="./myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='salary-predictor',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
print(service.swagger_uri)

Creating image
Image creation operation finished for image salary-predictor:4, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"
http://52.180.92.69:80/score
http://52.180.92.69:80/swagger.json
Wall time: 3min 35s


## 4. Swagger is Generated Based on the Provided Decorator

In [29]:
%%time
import json
import requests
from pprint import pprint

headers = {'Content-Type': 'application/json'}
print(json.dumps(requests.get(service.swagger_uri, headers=headers).json(), indent=4))

{
    "swagger": "2.0",
    "info": {
        "title": "salary-predictor",
        "description": "API specification for the Azure Machine Learning service salary-predictor",
        "version": "1.0"
    },
    "schemes": [
        "https"
    ],
    "consumes": [
        "application/json"
    ],
    "produces": [
        "application/json"
    ],
    "securityDefinitions": {
        "Bearer": {
            "type": "apiKey",
            "name": "Authorization",
            "in": "header",
            "description": "For example: Bearer abc123"
        }
    },
    "paths": {
        "/": {
            "get": {
                "operationId": "ServiceHealthCheck",
                "description": "Simple health check endpoint to ensure the service is up at any given point.",
                "responses": {
                    "200": {
                        "description": "If service is up and running, this response will be returned with the content 'Healthy'",
                        "sc

## 5. Run the Service

In [30]:
%%time
import json
import numpy as np

run_input = {'data': [[1.8]]}

result = service.run(json.dumps(run_input))

print(np.array(result))

[43638.88864165]
Wall time: 51.9 ms


## 6. Clean Up

In [31]:
service.delete()